In [18]:
## pip install moviepy==1.0.0

import os
import librosa
import pandas as pd
from moviepy.editor import *
from tqdm import tqdm_notebook
import numpy as np
import re

In [19]:
#from utils import make_dir
def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

In [20]:
background_dir = './data/audio_label_clip/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성

In [21]:
audio_label_clip_dir = os.path.join('.', 'data', 'audio_label_clip')
make_dir(audio_label_clip_dir)

In [31]:
def save(audios, label):
    data = pd.DataFrame({
        'audios':audios, 'label':label })
    data.to_csv(audio_label_clip_dir+'/audio_label_clip.csv', index=False, encoding='euc-kr')

In [34]:
## 레이블링 데이터 위치 : ./data/label
files = []
for file in os.listdir(os.path.join('.', 'data', 'label_3')):
    _, file_id = os.path.split(file)
    if file_id.endswith('txt'):
        name = file_id.split(".")[0]
        if(name not in files):
            files.append(name)
print(files)

background_dir = './data/audio_label_clip/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성print(files)


['khT_BIS7l8A_0', 'khT_BIS7l8A_1', 'khT_BIS7l8A_10', 'khT_BIS7l8A_11', 'khT_BIS7l8A_12', 'khT_BIS7l8A_13', 'khT_BIS7l8A_14', 'khT_BIS7l8A_15', 'khT_BIS7l8A_16', 'khT_BIS7l8A_17', 'khT_BIS7l8A_18', 'khT_BIS7l8A_19', 'khT_BIS7l8A_2', 'khT_BIS7l8A_20', 'khT_BIS7l8A_21', 'khT_BIS7l8A_22', 'khT_BIS7l8A_23', 'khT_BIS7l8A_24', 'khT_BIS7l8A_25', 'khT_BIS7l8A_26', 'khT_BIS7l8A_3', 'khT_BIS7l8A_4', 'khT_BIS7l8A_5', 'khT_BIS7l8A_6', 'khT_BIS7l8A_7', 'khT_BIS7l8A_8', 'khT_BIS7l8A_9', 'KS5wiED6vdI_0', 'KS5wiED6vdI_1', 'KS5wiED6vdI_10', 'KS5wiED6vdI_11', 'KS5wiED6vdI_12', 'KS5wiED6vdI_13', 'KS5wiED6vdI_14', 'KS5wiED6vdI_2', 'KS5wiED6vdI_3', 'KS5wiED6vdI_4', 'KS5wiED6vdI_5', 'KS5wiED6vdI_6', 'KS5wiED6vdI_7', 'KS5wiED6vdI_8', 'KS5wiED6vdI_9', 'KWlbz7VyIb4_0', 'KWlbz7VyIb4_1', 'KWlbz7VyIb4_10', 'KWlbz7VyIb4_11', 'KWlbz7VyIb4_12', 'KWlbz7VyIb4_13', 'KWlbz7VyIb4_2', 'KWlbz7VyIb4_3', 'KWlbz7VyIb4_4', 'KWlbz7VyIb4_5', 'KWlbz7VyIb4_6', 'KWlbz7VyIb4_7', 'KWlbz7VyIb4_8', 'KWlbz7VyIb4_9', 'L5AAD7w7_mc_0', 'L5A

In [14]:
background_dir = './data/audio_label_clip_for_test/background'  # 배경 디렉터리 경로
os.makedirs(background_dir, exist_ok=True)  # 배경 디렉터리 생성

activate_dir = './data/audio_label_clip_for_test/activate'  # 배경 디렉터리 경로
os.makedirs(activate_dir, exist_ok=True)  # 배경 디렉터리 생성

negative_dir = './data/audio_label_clip_for_test/negative'  # 배경 디렉터리 경로
os.makedirs(negative_dir, exist_ok=True)  # 배경 디렉터리 생성print(files)

In [36]:
audio = []
label = []
#라벨링
shi = 0
jon = 0
sha = 0
byoung = 0
sang = 0 
do = 0
mi = 0
gae = 0
jot = 0
background = 0
negative = 0
count = 0
x = 0

# 이제 욕 추가될때마다 여기 counter 디렉터리에 추가해주면 됨
counter = {
    10: shi,
    11: jon,
    12: sha,
    13: byoung,
    14: do,
    15: sang,
    16: mi,
    17: gae,
    18: jot,
    8: x,
    9: background,
    0: negative
}

for file in files:
    try: 
        print("file name:" + file)
        text = './data/label_3/' + file + ".txt"
        aud = './data/label_3/' + file + ".wav"
        # 정규표현식 패턴
        pattern = r"^(.*?)_\d+$"
        # 정규표현식 매칭
        match = re.match(pattern, file)
        raw_aud = './data/audio/' + str(match.group(1)) + ".wav"

        print("read new file")
        data = pd.read_csv(text, sep="\t", names=['start', 'end', 'label'])
        print(data)
        for i in range(len(data)):
            lab = data.label[i]
            print(data.start[i], data.end[i], data.label[i])
            try:
                start = float(data.start[i])
                end = float(data.end[i])
            except ValueError:
                continue
            if start >= end:
                continue
            if np.isnan(lab):
                continue
            full_audio = AudioFileClip(raw_aud)

            # 각 라벨 이름 정의
            lab_name = {
                10: "shi",
                11: "jon",
                12: "sha",
                13: "byoung",
                14: "do",
                15: "sang",
                16: "mi",
                17: "gae",
                18: "jot",
                8: "x",
                9: "background",
                0: "negative"
            }[lab]
            
            lab_counter = counter[lab]
            name = f"{lab_name}_{lab_counter}"
            counter[lab] += 1

            tmp_audio = full_audio.subclip(start, end)
            
            #라벨에 맞게 디렉터리 경로 초기화해주고
            if lab == 10 or lab == 11 or lab == 12 or lab == 13 or lab == 14 or lab == 15 or lab == 16 or lab == 17 or lab == 18:
                directory = activate_dir
            elif lab == 9:
                directory = background_dir
            elif lab == 0:
                directory = negative_dir
            else:
                continue
            # 라벨에 맞는 디렉터리에 저장
            name = os.path.join(directory, name + ".wav")
            try:
                tmp_audio.write_audiofile(name)
            except OSError:
                print('oserror')
                continue
                
            audio.append(name)
            label.append(int(lab))
            print('완료됬습니다.')
        count += 1
    except Exception as e:
        pass

save(audio, label)

file name:khT_BIS7l8A_0
read new file
   start  end  label
0    2.5  7.4     10
2.5 7.4 10
MoviePy - Writing audio in ./data/audio_label_clip/activate\shi_0.wav


MoviePy - Done.
완료됬습니다.
file name:khT_BIS7l8A_1
read new file
   start    end  label
0    7.4  11.31     17
7.4 11.31 17


MoviePy - Writing audio in ./data/audio_label_clip/activate\gae_0.wav
MoviePy - Done.
완료됬습니다.
file name:khT_BIS7l8A_10
read new file
   start   end  label
0   39.0  44.3     17
39.0 44.3 17


MoviePy - Writing audio in ./data/audio_label_clip/activate\gae_1.wav
MoviePy - Done.
완료됬습니다.
file name:khT_BIS7l8A_11
read new file
   start   end  label
0   44.3  49.0     10
44.3 49.0 10


MoviePy - Writing audio in ./data/audio_label_clip/activate\shi_1.wav
MoviePy - Done.
완료됬습니다.
file name:khT_BIS7l8A_12
read new file
   start   end  label
0   49.0  51.4     10
49.0 51.4 10


...


완료됬습니다.
file name:MS3OADFiWVs_7
read new file
   start     end  label
0   24.7  25.199     12
24.7 25.199 12
MoviePy - Writing audio in ./data/audio_label_clip/activate\sha_9.wav


MoviePy - Done.
완료됬습니다.
file name:MS3OADFiWVs_8
read new file
    start    end  label
0  25.199  29.58      0
25.199 29.58 0
MoviePy - Writing audio in ./data/audio_label_clip/negative\negative_98.wav


chunk:   0%|          | 0/86 [00:00<?, ?it/s, now=None]          

MoviePy - Done.
완료됬습니다.
file name:MS3OADFiWVs_9
read new file
   start    end  label
0  29.58  33.44     12
29.58 33.44 12
MoviePy - Writing audio in ./data/audio_label_clip/activate\sha_10.wav


MoviePy - Done.
완료됬습니다.
